<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Creating and Provisioning Storage with Python

## Azure Actions Covered

* Creating storage resources


In this lecture, we're going to take a look at how to create storage resources via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add our new storage import as well.

In [1]:
# For generating storage account name
import random

from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
# New import
from azure.mgmt.storage import StorageManagementClient

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We need to instantiate our Azure credential as well as our `ResourceManagementClient()` object.

In [2]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, AZURE_SUBSCRIPTION_ID)

We'll need to create our Storage objects under a resource group, so let's create a default Resource Group in a default Azure location if one doesn't exist yet.

The `create_or_update()` method takes the following parameters:
* `resource_group_name` - Name of the resource group to create or update
* `parameters` - Dict of parameters or `ResourceGroup` object. For full list of parameters, see the [ResourceGroup class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-resource/azure.mgmt.resource.resources.v2022_09_01.models.resourcegroup?view=azure-python)

In [4]:
if not resource_client.resource_groups.get(DEFAULT_RESOURCE_GROUP):
    rg_result = resource_client.resource_groups.create_or_update(
        resource_group_name=DEFAULT_RESOURCE_GROUP,
        parameters={
            'location': DEFAULT_LOCATION
        }
    )

We can instantiate a `StorageManagementClient()` object using the same pattern as with `ResourceManagementClient()`.

In [3]:
storage_client = StorageManagementClient(credential, AZURE_SUBSCRIPTION_ID)

As we saw in the previous lecture, storage accounts must have unique names since they're used in our URLs. We'll use a `while` loop to try out variations on a base name for our account until we find one that works.
* First, use `storage_client.storage_accounts.check_name_availability()` to check whether the generated name is available.
    * `account_name` - This is a `StorageAccountCheckNameAvailabilityParameters` model. For full list of parameters, see the [StorageAccountCheckNameAvailabilityParameters class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-storage/azure.mgmt.storage.v2022_09_01.models.storageaccountchecknameavailabilityparameters?view=azure-python)
* If the name isn't available, generate a new one.
* If the name is available, use `begin_create()` to create the account. You can specify options like we did in the CLI.
    * `location` - Azure location
    * `kind` - Indicates the type of storage account, such as `StorageV2`, `FileStorage`, or `BlobStorage`.
    * `sku` - Dictionary of SKU options for storage account, such as the name of the SKU, including `Standard_LRS` (locally-redundant storage), `Standard_GRS` (geo-redundant storage), or `Standard_ZRS` (zone-redundant storage).
    * `tags` - Dictionary of key-value pairs for the resource.
    
For full list of parameters, see the [StorageAccountCreateParameters class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-storage/azure.mgmt.storage.v2022_09_01.models.storageaccountcreateparameters?view=azure-python)

In [15]:
STORAGE_ACCOUNT_BASE = 'benbstorage'
while True:
    account_name = f'{STORAGE_ACCOUNT_BASE}{random.randint(1,1000)}'
    if not storage_client.storage_accounts.check_name_availability({'name':account_name}).name_available:
        print(f'Account name {account_name} not available, trying again')
    else:
        print(f'Account name {account_name} available, creating account')
        poller =  storage_client.storage_accounts.begin_create(
            resource_group_name=DEFAULT_RESOURCE_GROUP,
            account_name=account_name,
            parameters={
                'location': DEFAULT_LOCATION,
                'kind': 'StorageV2',
                'sku': {
                    'name': 'Standard_LRS'
                }
            }
        )
        poller_result = poller.result()
        print(f'{poller_result.name} has been provisioned')
        break

Account name benbstorage226 available, creating account
benbstorage226 has been provisioned


In the next lecture, we'll see how to create blob containers and work with blobs in Azure Storage.